In [79]:
import pandas as pd
import numpy as np
import datetime
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from ta.trend import SMAIndicator, EMAIndicator, MACD
from pandas_datareader import data as web
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv

# ------------------------
# 1. Cargar clave desde .env
# ------------------------
load_dotenv()
api_key = os.getenv("FRED_API_KEY")

if not api_key:
    raise ValueError("❌ API Key de FRED no encontrada. Asegúrate de tener FRED_API_KEY en tu archivo .env")

# ------------------------
# 2. Cargar TLT + features técnicas
# ------------------------

df = pd.read_csv("/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/trading/project_trading_DT/data/raw/TLT.csv", parse_dates=["Date"])
df.sort_values("Date", inplace=True)

df["return_daily"] = df["Close"].pct_change()
for lag in range(1, 6):
    df[f"return_lag_{lag}"] = df["return_daily"].shift(lag)

df["rolling_std_return_5"] = df["return_daily"].rolling(window=5).std()
df["return_volatility_ratio"] = df["return_daily"] / df["rolling_std_return_5"]

df["price_diff"] = df["Close"] - df["Open"]
df["pct_diff"] = df["price_diff"] / df["Open"]
df["log_vol"] = np.log(df["Volume"] + 1)

df["sma_5"] = SMAIndicator(close=df["Close"], window=5).sma_indicator()
df["ema_5"] = EMAIndicator(close=df["Close"], window=5).ema_indicator()
df["rsi_5"] = RSIIndicator(close=df["Close"], window=5).rsi()
macd = MACD(close=df["Close"])
df["macd"] = macd.macd()
df["macd_signal"] = macd.macd_signal()
df["macd_above_signal"] = (df["macd"] > df["macd_signal"]).astype(int)

df["day_of_week"] = df["Date"].dt.dayofweek
df["month"] = df["Date"].dt.month
df["is_month_end"] = df["Date"].dt.is_month_end.astype(int)

# ------------------------
# 3. Datos FRED (macroeconómicos)
# ------------------------

start = df["Date"].min()
end = df["Date"].max()

fred_series = {
    "GS10": "us10y_yield",           # Tasa bono 10 años
    "FEDFUNDS": "fed_rate",          # Tasa Fed
    "CPIAUCSL": "cpi",               # Inflación
    "UNRATE": "unemployment_rate"    # Desempleo
}

macro_df = pd.DataFrame()

for code, name in fred_series.items():
    try:
        temp = web.DataReader(code, "fred", start, end, api_key=api_key)
        temp = temp.rename(columns={code: name}).reset_index()
        print(f"✅ {name} descargado con {len(temp)} registros")
        
        if macro_df.empty:
            macro_df = temp
        else:
            macro_df = macro_df.merge(temp, on="DATE", how="outer")
    
    except Exception as e:
        print(f"❌ Error al descargar {code} ({name}): {e}")


macro_df.rename(columns={"DATE": "Date"}, inplace=True)

✅ us10y_yield descargado con 171 registros
✅ fed_rate descargado con 171 registros
✅ cpi descargado con 171 registros
✅ unemployment_rate descargado con 171 registros


In [80]:
df

,Date,Close,High,Low,Open,Volume,return_daily,return_lag_1,return_lag_2,return_lag_3,...,log_vol,sma_5,ema_5,rsi_5,macd,macd_signal,macd_above_signal,day_of_week,month,is_month_end
0,2010-12-27,63.565655,63.647007,62.575832,62.887693,5285900,NaN,NaN,NaN,NaN,...,15.480554,NaN,NaN,NaN,NaN,NaN,0,0,12,0
1,2010-12-28,62.320736,63.313949,62.116649,63.218709,13972100,-0.019585,NaN,NaN,NaN,...,16.452573,NaN,NaN,NaN,NaN,NaN,0,1,12,0
2,2010-12-29,63.375168,63.620065,62.225488,62.422771,13115400,0.016919,-0.019585,NaN,NaN,...,16.389298,NaN,NaN,NaN,NaN,NaN,0,2,12,0
3,2010-12-30,63.327553,63.456808,62.864965,63.184694,7029900,-0.000751,0.016919,-0.019585,NaN,...,15.765683,NaN,NaN,NaN,NaN,NaN,0,3,12,0
4,2010-12-31,64.028198,64.422758,63.565606,63.613225,8516000,0.011064,-0.000751,0.016919,-0.019585,...,15.957457,63.323462,63.515749,67.064867,NaN,NaN,0,4,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591,2025-04-07,90.050003,92.190002,89.800003,91.879997,124987600,-0.030156,0.010887,0.004594,-0.000656,...,18.643725,91.534000,91.144490,39.674615,0.464259,0.373795,1,0,4,0
3592,2025-04-08,88.349998,90.199997,88.220001,89.279999,73861900,-0.018878,-0.030156,0.010887,0.004594,...,18.117708,90.906000,90.212993,28.277143,0.235998,0.346235,0,1,4,0
3593,2025-04-09,88.870003,88.910004,85.889999,87.269997,131353500,0.005886,-0.018878,-0.030156,0.010887,...,18.693403,90.394000,89.765329,35.375530,0.095953,0.296179,0,2,4,0
3594,2025-04-10,86.419998,88.180000,86.349998,87.739998,78592000,-0.027568,0.005886,-0.018878,-0.030156,...,18.179780,89.308000,88.650219,22.348973,-0.210304,0.194882,0,3,4,0


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3596 entries, 0 to 3595
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     3596 non-null   datetime64[ns]
 1   Close                    3596 non-null   float64       
 2   High                     3596 non-null   float64       
 3   Low                      3596 non-null   float64       
 4   Open                     3596 non-null   float64       
 5   Volume                   3596 non-null   int64         
 6   return_daily             3595 non-null   float64       
 7   return_lag_1             3594 non-null   float64       
 8   return_lag_2             3593 non-null   float64       
 9   return_lag_3             3592 non-null   float64       
 10  return_lag_4             3591 non-null   float64       
 11  return_lag_5             3590 non-null   float64       
 12  rolling_std_return_5     3591 non-

In [82]:
df = df.merge(macro_df, on="Date", how="left")

In [83]:
df

,Date,Close,High,Low,Open,Volume,return_daily,return_lag_1,return_lag_2,return_lag_3,...,macd,macd_signal,macd_above_signal,day_of_week,month,is_month_end,us10y_yield,fed_rate,cpi,unemployment_rate
0,2010-12-27,63.565655,63.647007,62.575832,62.887693,5285900,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,12,0,NaN,NaN,NaN,NaN
1,2010-12-28,62.320736,63.313949,62.116649,63.218709,13972100,-0.019585,NaN,NaN,NaN,...,NaN,NaN,0,1,12,0,NaN,NaN,NaN,NaN
2,2010-12-29,63.375168,63.620065,62.225488,62.422771,13115400,0.016919,-0.019585,NaN,NaN,...,NaN,NaN,0,2,12,0,NaN,NaN,NaN,NaN
3,2010-12-30,63.327553,63.456808,62.864965,63.184694,7029900,-0.000751,0.016919,-0.019585,NaN,...,NaN,NaN,0,3,12,0,NaN,NaN,NaN,NaN
4,2010-12-31,64.028198,64.422758,63.565606,63.613225,8516000,0.011064,-0.000751,0.016919,-0.019585,...,NaN,NaN,0,4,12,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591,2025-04-07,90.050003,92.190002,89.800003,91.879997,124987600,-0.030156,0.010887,0.004594,-0.000656,...,0.464259,0.373795,1,0,4,0,NaN,NaN,NaN,NaN
3592,2025-04-08,88.349998,90.199997,88.220001,89.279999,73861900,-0.018878,-0.030156,0.010887,0.004594,...,0.235998,0.346235,0,1,4,0,NaN,NaN,NaN,NaN
3593,2025-04-09,88.870003,88.910004,85.889999,87.269997,131353500,0.005886,-0.018878,-0.030156,0.010887,...,0.095953,0.296179,0,2,4,0,NaN,NaN,NaN,NaN
3594,2025-04-10,86.419998,88.180000,86.349998,87.739998,78592000,-0.027568,0.005886,-0.018878,-0.030156,...,-0.210304,0.194882,0,3,4,0,NaN,NaN,NaN,NaN


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3596 entries, 0 to 3595
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     3596 non-null   datetime64[ns]
 1   Close                    3596 non-null   float64       
 2   High                     3596 non-null   float64       
 3   Low                      3596 non-null   float64       
 4   Open                     3596 non-null   float64       
 5   Volume                   3596 non-null   int64         
 6   return_daily             3595 non-null   float64       
 7   return_lag_1             3594 non-null   float64       
 8   return_lag_2             3593 non-null   float64       
 9   return_lag_3             3592 non-null   float64       
 10  return_lag_4             3591 non-null   float64       
 11  return_lag_5             3590 non-null   float64       
 12  rolling_std_return_5     3591 non-

In [86]:
# ------------------------
# 1. Rellenar variables macroeconómicas con forward fill
# ------------------------

macro_cols = ["us10y_yield", "fed_rate", "cpi", "unemployment_rate"]
df[macro_cols] = df[macro_cols].ffill()

# ------------------------
# 2. Crear variable objetivo (target): sube el cierre al día siguiente
# ------------------------

df["target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)

# ------------------------
# 3. Seleccionar columnas necesarias
# ------------------------

selected_features = [
    "day_of_week", "is_month_end", "month",
    "price_diff", "pct_diff", "log_vol",
    "return_lag_1", "return_lag_2", "return_lag_3", "return_lag_4", "return_lag_5",
    "sma_5", "ema_5", "rolling_std_return_5",
    "rsi_5", "macd", "macd_signal", "macd_above_signal",
    "return_volatility_ratio",
    "us10y_yield", "fed_rate", "cpi", "unemployment_rate"
]

df.dropna(inplace=True)


print(f"✅ Dataset final listo con {len(df)} filas")



✅ Dataset final listo con 3563 filas


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3563 entries, 33 to 3595
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     3563 non-null   datetime64[ns]
 1   Close                    3563 non-null   float64       
 2   High                     3563 non-null   float64       
 3   Low                      3563 non-null   float64       
 4   Open                     3563 non-null   float64       
 5   Volume                   3563 non-null   int64         
 6   return_daily             3563 non-null   float64       
 7   return_lag_1             3563 non-null   float64       
 8   return_lag_2             3563 non-null   float64       
 9   return_lag_3             3563 non-null   float64       
 10  return_lag_4             3563 non-null   float64       
 11  return_lag_5             3563 non-null   float64       
 12  rolling_std_return_5     3563 non-null

In [88]:
# 10) Guardar CSV con todas las features
output_path = "/Users/melaniealvarez/Documents/Octavo semestres/Data Mining/trading/project_trading_DT/data/processed/TLT_clean.csv"
df.to_csv(output_path, index=False)
print(f"Features saved to {output_path}")

Features saved to /Users/melaniealvarez/Documents/Octavo semestres/Data Mining/trading/project_trading_DT/data/processed/TLT_clean.csv
